# 📊 PyKwant Tutorial: Performance & Analytics

Questo notebook introduce il modulo `pykwant.analytics`, progettato per misurare la performance finanziaria di asset o portafogli.

A differenza dei moduli precedenti focalizzati sul *Pricing* (valutazione ex-ante), questo modulo si concentra sull'*Analisi* di serie storiche (valutazione ex-post).

Copriremo:
1. Calcolo dei **Rendimenti** (Semplici e Logaritmici).
2. Stima della **Volatilità** Annualizzata.
3. Metriche Risk-Adjusted: **Sharpe Ratio** e **Sortino Ratio**.
4. Analisi del Rischio Estremo: **Maximum Drawdown**.

## 1. Setup e Generazione Dati

Poiché `pykwant` è una libreria zero-dependency, non usiamo pandas per caricare dati. Simuleremo invece una serie di prezzi usando la libreria standard `random` per creare un "Random Walk" con un trend positivo.

In [1]:
import math
import random

from pykwant import analytics

# Configurazione Seed per riproducibilità
random.seed(42)

# 1. Generiamo una serie di prezzi sintetici (1 Anno Trading = 252 giorni)
initial_price = 100.0
prices = [initial_price]

# Parametri simulazione: Drift annuo 10%, Volatilità annua 20%
mu_daily = 0.10 / 252
sigma_daily = 0.20 / math.sqrt(252)

for _ in range(252):
    # Geometric Brownian Motion step approssimato
    shock = random.gauss(0, 1)
    ret = mu_daily + sigma_daily * shock
    new_price = prices[-1] * (1 + ret)
    prices.append(new_price)

print(f"Serie generata: {len(prices)} prezzi.")
print(f"Start: {prices[0]:.2f}")
print(f"End:   {prices[-1]:.2f}")


# Helper per formattare l'output
def print_metric(label, value, is_pct=False, decimals=4):
    if is_pct:
        fmt = f"{{:.{decimals - 2}%}}"
    else:
        fmt = f"{{:.{decimals}f}}"
    print(f"{label:<25}: {fmt.format(value)}")

Serie generata: 253 prezzi.
Start: 100.00
End:   149.85


## 2. Calcolo dei Rendimenti

Possiamo calcolare sia i rendimenti discreti (semplici) che quelli continui (logaritmici).

* **Simple**: $(P_t - P_{t-1}) / P_{t-1}$
* **Log**: $\ln(P_t / P_{t-1})$

In [2]:
# Calcolo Rendimenti
simple_rets = analytics.simple_returns(prices)
log_rets = analytics.log_returns(prices)

print("--- Analisi Rendimenti (Primi 5 giorni) ---")
for i in range(5):
    print(
        f"Day {i + 1}: Price={prices[i + 1]:.2f} | Simple={simple_rets[i]:.4%} | Log={log_rets[i]:.4%}"
    )

--- Analisi Rendimenti (Primi 5 giorni) ---
Day 1: Price=99.86 | Simple=-0.1419% | Log=-0.1420%
Day 2: Price=99.68 | Simple=-0.1782% | Log=-0.1783%
Day 3: Price=99.58 | Simple=-0.1006% | Log=-0.1006%
Day 4: Price=100.50 | Simple=0.9241% | Log=0.9199%
Day 5: Price=100.38 | Simple=-0.1211% | Log=-0.1211%


## 3. Volatilità

La volatilità è la misura standard del rischio. La funzione `annualized_volatility` calcola la deviazione standard dei rendimenti e la scala per $\sqrt{252}$.

In [3]:
vol_ann = analytics.annualized_volatility(simple_rets, periods_per_year=252)

print("--- Rischio ---")
print_metric("Volatilità Annualizzata", vol_ann, is_pct=True)
# Ci aspettiamo un valore vicino al 20% (0.20) impostato nella simulazione

--- Rischio ---
Volatilità Annualizzata  : 19.52%


## 4. Performance Risk-Adjusted (Sharpe & Sortino)

Il rendimento assoluto non racconta tutta la storia. Dobbiamo aggiustarlo per il rischio assunto.

* **Sharpe Ratio**: Premia il rendimento in eccesso rispetto al tasso privo di rischio, penalizzando la volatilità totale.
* **Sortino Ratio**: Simile allo Sharpe, ma penalizza solo la **Downside Volatility** (volatilità negativa). È utile per strategie che hanno volatilità positiva (es. trend following).

In [4]:
risk_free_rate = 0.02  # Assumiamo un tasso risk-free del 2% annuo

sharpe = analytics.sharpe_ratio(simple_rets, risk_free_rate=risk_free_rate)
sortino = analytics.sortino_ratio(simple_rets, target_return=risk_free_rate)

print("--- Metriche Risk-Adjusted ---")
print_metric("Risk Free Rate", risk_free_rate, is_pct=True)
print_metric("Sharpe Ratio", sharpe)
print_metric("Sortino Ratio", sortino)

if sortino > sharpe:
    print(
        "\nNota: Il Sortino è maggiore dello Sharpe, indicando che parte della volatilità "
        "era 'buona' (rialzista)."
    )

--- Metriche Risk-Adjusted ---
Risk Free Rate           : 2.00%
Sharpe Ratio             : 2.0685
Sortino Ratio            : 3.2541

Nota: Il Sortino è maggiore dello Sharpe, indicando che parte della volatilità era 'buona' (rialzista).


## 5. Maximum Drawdown (MDD)

Il Max Drawdown misura la peggiore perdita possibile (dal picco al minimo successivo) che un investitore avrebbe subito comprando al massimo e vendendo al minimo.

In [5]:
mdd = analytics.max_drawdown(prices)

print("--- Worst Case Scenario ---")
print_metric("Maximum Drawdown", mdd, is_pct=True)

print(
    f"\nInterpretazione: Nel momento peggiore, l'asset ha perso il {mdd:.2%} dal suo picco "
    "precedente."
)

--- Worst Case Scenario ---
Maximum Drawdown         : 7.53%

Interpretazione: Nel momento peggiore, l'asset ha perso il 7.53% dal suo picco precedente.
